In [6]:
### IMPORTS ###
import os
import xml.etree.ElementTree as ET
from PIL import Image
import shutil
import random
from ultralytics import YOLO
import cv2

In [ ]:
## INSTRUCTIONS:
# Download dataset from: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection
# unzip and rename to 'car-license-plate-detection' then add to 'segment' directory 
# Run this cell

annotation_folder = "car-license-plate-detection/annotations"
output_folder = "car-license-plate-detection/yolo_annotations" 
os.makedirs(output_folder, exist_ok=True)

# Only one class in this dataset
class_map = {'licence':0}

In [ ]:
# Annotations are in PASCAL VOC format.
# Need to convert to YOLO format
    """ Parses xml annotations and converts them to YOLO format
    """

def convert_annotation(xml_file, output_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Image dimensions from <size> tag
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    with open(output_file, 'w') as out_file:
        for obj in root.iter('object'):
            label = obj.find('name').text.lower()
            
            # This should be unecessary, but safety first :)
            if label not in class_map:
                continue
            
            # YOLO needs class id
            class_id = class_map[label]

            # Get bbox coords
            xmlbox = obj.find('bndbox')
            xmin = float(xmlbox.find('xmin').text)
            ymin = float(xmlbox.find('ymin').text)
            xmax = float(xmlbox.find('xmax').text)
            ymax = float(xmlbox.find('ymax').text)

            # YOLO requires normalized values (x,y,w,h)
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height

            # Write new file in YOLO format
            out_file.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

for xml_file in os.listdir(annotation_folder):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(annotation_folder, xml_file)
        #Cars0.xml -> Cars0.txt
        output_file = os.path.join(output_folder, os.path.splitext(xml_file)[0] + '.txt')
        convert_annotation(xml_path, output_file)

print("Finished converting. Output directory:", output_folder)

Finished converting. Output directory: car-license-plate-detection/yolo_annotations


In [ ]:
# Need to split dataset

# Base directories 
base_images_dir = 'car-license-plate-detection/images'
base_xml_dir = 'car-license-plate-detection/annotations'         
base_yolo_dir = 'car-license-plate-detection/yolo_annotations'     

# New Directories 
train_images_dir = 'car-license-plate-detection/train/images'
val_images_dir   = 'car-license-plate-detection/val/images'
test_images_dir  = 'car-license-plate-detection/test/images'

# Ultralytics expects /labels directory
train_yolo_dir   = 'car-license-plate-detection/train/labels'
val_yolo_dir     = 'car-license-plate-detection/val/labels'
test_yolo_dir    = 'car-license-plate-detection/test/labels'

train_xml_dir    = 'car-license-plate-detection/train/xml_annotations'
val_xml_dir      = 'car-license-plate-detection/val/xml_annotations'  
test_xml_dir     = 'car-license-plate-detection/test/xml_annotations' 

# Create destination directories
dest_dirs = [train_images_dir, val_images_dir, test_images_dir,
             train_yolo_dir, val_yolo_dir, test_yolo_dir,
             train_xml_dir, val_xml_dir, test_xml_dir]
for d in dest_dirs:
    os.makedirs(d, exist_ok=True)

# List all image files 
image_files = [f for f in os.listdir(base_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(image_files)

#(70% train, 15% validation, 15% test)
n_total = len(image_files)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val 

train_files = image_files[:n_train]
val_files   = image_files[n_train:n_train+n_val]
test_files  = image_files[n_train+n_val:]

def copy_files(file_list, src_img_dir, src_yolo_dir, src_xml_dir, dst_img_dir, dst_yolo_dir, dst_xml_dir):
    for file in file_list:
        # Copy image file
        shutil.copy(os.path.join(src_img_dir, file), os.path.join(dst_img_dir, file))
        
        # YOLO ANNOTATIONS
        yolo_file = os.path.splitext(file)[0] + '.txt'
        src_yolo_file = os.path.join(src_yolo_dir, yolo_file)
        if os.path.exists(src_yolo_file):
            shutil.copy(src_yolo_file, os.path.join(dst_yolo_dir, yolo_file))
        
        # XML ANNOTATIONS
        xml_file = os.path.splitext(file)[0] + '.xml'
        src_xml_file = os.path.join(src_xml_dir, xml_file)
        if os.path.exists(src_xml_file):
            shutil.copy(src_xml_file, os.path.join(dst_xml_dir, xml_file))

# Training files
copy_files(train_files, base_images_dir, base_yolo_dir, base_xml_dir, 
           train_images_dir, train_yolo_dir, train_xml_dir)

# Validation files
copy_files(val_files, base_images_dir, base_yolo_dir, base_xml_dir, 
           val_images_dir, val_yolo_dir, val_xml_dir)

# Test Files
copy_files(test_files, base_images_dir, base_yolo_dir, base_xml_dir, 
           test_images_dir, test_yolo_dir, test_xml_dir)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data='data.yaml',
    epochs=20,
    batch=16,
    imgsz=640 # Yolo assumes 640
)

In [ ]:
###
# This code takes license plate photos then uses our model to crop images to just the licesnse plate
# and save the cropped images.
###

# Change after training
model = YOLO('runs/detect/train7/weights/best.pt')

input_dir = 'car-license-plate-detection/images'
output_dir = 'cropped_images'

os.makedirs(output_dir, exist_ok=True)

for image in os.listdir(input_dir):
    if image.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_dir, image)
        
        # Inference
        results = model(image_path)
        result = results[0]
        img = result.orig_img
        
        for idx, bbox in enumerate(result.boxes.xyxy):
            x1, y1, x2, y2 = map(int, bbox)
            
            cropped_img = img[y1:y2, x1:x2]
            
            org_name = os.path.splitext(image)[0]
            new_name = f"{org_name}_cropped.png"
            output_path = os.path.join(output_dir, new_name)
            
            cv2.imwrite(output_path, cropped_img)
            
# Compress output folder
shutil.make_archive('cropped_images', 'zip', output_dir)


image 1/1 /Users/aidanmacnichol/Documents/uofc/ENEL645/ENEL645_Project_Group2/segment/car-license-plate-detection/images/Cars207.png: 640x544 1 license_plate, 141.7ms
Speed: 3.2ms preprocess, 141.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /Users/aidanmacnichol/Documents/uofc/ENEL645/ENEL645_Project_Group2/segment/car-license-plate-detection/images/Cars213.png: 480x640 1 license_plate, 160.7ms
Speed: 1.4ms preprocess, 160.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/aidanmacnichol/Documents/uofc/ENEL645/ENEL645_Project_Group2/segment/car-license-plate-detection/images/Cars53.png: 480x640 1 license_plate, 108.9ms
Speed: 1.1ms preprocess, 108.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)



[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


image 1/1 /Users/aidanmacnichol/Documents/uofc/ENEL645/ENEL645_Project_Group2/segment/car-license-plate-detection/images/Cars47.png: 480x640 2 license_plates, 115.7ms
Speed: 3.1ms preprocess, 115.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/aidanmacnichol/Documents/uofc/ENEL645/ENEL645_Project_Group2/segment/car-license-plate-detection/images/Cars90.png: 480x640 1 license_plate, 107.5ms
Speed: 1.3ms preprocess, 107.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/aidanmacnichol/Documents/uofc/ENEL645/ENEL645_Project_Group2/segment/car-license-plate-detection/images/Cars159.png: 544x640 1 license_plate, 167.3ms
Speed: 1.5ms preprocess, 167.3ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /Users/aidanmacnichol/Documents/uofc/ENEL645/ENEL645_Project_Group2/segment/car-license-plate-detection/images/Cars84.png: 480x640 1 license_plate, 107.9ms
Speed: 1.8ms preprocess, 107.9ms infe